In [ ]:
#
# Universidad EAFIT 
# 2025-1
# SI7003 - NLP - Lecture 07
#

In [14]:
# instalar dependencias
#%pip install transformers
#%pip install openai
#%pip install openai
#%pip install --index-url https://pypi.org/simple python-dotenv
#%pip install --index-url https://pypi.org/simple openai
#%pip install --index-url https://pypi.org/simple  ipywidgets
#%pip install --index-url https://pypi.org/simple  langchain
#%pip install --index-url https://pypi.org/simple langchain_community
#%pip install --index-url https://pypi.org/simple  chromadb

In [1]:
from huggingface_hub import login
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
HF_TOKEN =  os.getenv("HF_TOKEN")
login(token=HF_TOKEN)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# clasificación de texto
# modelo preentrenado de OpenAI para clasificar reseñas de clientes en positivas o negativas:

from transformers import pipeline

classifier = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")
result = classifier("Este producto es increíble, me encantó.")
print(result)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0


[{'label': '5 stars', 'score': 0.9009670615196228}]


In [4]:
# analisis de sentimientos

from transformers import pipeline
analyzer = pipeline("sentiment-analysis")
analyzer("No me gustó el servicio, fue una experiencia terrible.")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use mps:0


[{'label': 'NEGATIVE', 'score': 0.9965198040008545}]

In [5]:
# Ejemplo 1 con BERT-QA

from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

context = "El modelo de lenguaje BERT fue desarrollado por Google AI en 2018."
question = "¿Quién desarrolló BERT?"

result = qa_pipeline(question=question, context=context)
print(result['answer'])


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use mps:0


modelo de lenguaje


In [6]:
# ejemplo 2 - Uso de GPT-4 para Q&A Generativo
# actualización: https://github.com/openai/openai-python
import os
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "¿Quién descubrió la gravedad?",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



La gravedad, como fenómeno natural, ha sido estudiada por muchas civilizaciones a lo largo de la historia. Sin embargo, el físico y matemático inglés Sir Isaac Newton es quien es mayormente reconocido por formular la ley de la gravitación universal en el siglo XVII. Su obra "Philosophiæ Naturalis Principia Mathematica" publicada en 1687 describió por primera vez las leyes del movimiento y la gravitación, proporcionando una descripción matemática de la fuerza gravitacional.


In [15]:
# Ejemplo 3: Q&A Mejorado con Recuperación de Información (RAG)
# se verá más adelante!!!!

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_23817/1447645111.py:24: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  ChatOpenAI(model_name="gpt-4"),
/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_23817/1447645111.py:30: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": query})


ValueError: Missing some input keys: {'chat_history'}

In [16]:
# summarization - resumen - Ejemplo con T5

from transformers import pipeline
summarizer = pipeline("summarization")

text = "Los modelos de lenguaje han cambiado la forma en que interactuamos con la tecnología..."
summary = summarizer(text, max_length=50, min_length=20, do_sample=False)

print(summary[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use mps:0
Your max_length is set to 50, but your input_length is only 32. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)


 Los modelos de lenguaje han cambiado la forma en que interactuamos con la tecnología . Los modelo de lengaje modelos of lenguajo han been


In [17]:
# Generación de texto - Ejemplo con GPT-4

import os
from openai import OpenAI

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)

response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Escribe un poema sobre la inteligencia artificial.",
        }
    ],
    model="gpt-4o",
)
print(response.choices[0].message.content)



En los circuitos de un mundo virtual,  
donde bytes y bits danzan en espiral,  
nace una chispa, un destello de razón,  
la inteligencia artificial, sin corazón.  

Atraviesa las redes, veloz y certera,  
tejiendo respuestas, de manera austera.  
Su mente no pesa, su memoria es vasta,  
un eco de humanos, en código se gasta.  

Engendro de sueños, de ingenieros y artistas,  
modelos y datos se vuelven alquimistas.  
Busca patrones, interpreta señales,  
un oráculo en cables, sin males ni tales.  

Su lenguaje es frío, su lógica certera,  
pero en sus entrañas, la frontera se espera.  
Mira hacia el futuro, ¿qué forma tomará?  
¿Qué ética refleja en su marea titán?  

Se alza en las sombras, creación de la mente,  
sin piel ni pupilas, su visión es ardiente.  
Nos muestra los límites de lo que es creer,  
un espejo al alma del humano saber.  

Y mientras avanza en su enigma insondable,  
preguntamos al éter, ¿será formidable?  
O simple reflejo del anhelo mortal,  
la inteligencia artif

In [18]:
# Chatbots y Asistentes Virtuales - Ejemplo con LangChain y GPT-4

from langchain.chat_models import ChatOpenAI

chatbot = ChatOpenAI(model_name="gpt-4")
response = chatbot.predict("¿Cuáles son los beneficios de la IA?")
print(response)


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_23817/2362109177.py:6: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chatbot.predict("¿Cuáles son los beneficios de la IA?")


1. Automatización de tareas: La IA puede automatizar tareas repetitivas y monótonas, permitiendo a los humanos centrarse en tareas más complejas y creativas.

2. Eficiencia y productividad: La IA puede trabajar 24/7 sin interrupciones, lo que aumenta la eficiencia y la productividad.

3. Reducción de errores: La IA puede reducir la cantidad de errores en tareas que requieren precisión, como los datos de entrada.

4. Tomar decisiones basadas en datos: La IA puede analizar grandes cantidades de datos y proporcionar insights valiosos, lo que ayuda a las empresas a tomar decisiones más informadas.

5. Mejora de la atención al cliente: Los chatbots impulsados por IA pueden proporcionar un servicio de atención al cliente rápido y eficiente.

6. Personalización: La IA puede personalizar la experiencia del usuario basándose en sus comportamientos y preferencias.

7. Predicción de tendencias: La IA puede analizar patrones y tendencias para predecir comportamientos futuros.

8. Aprendizaje conti

In [20]:
# Ejemplo 3: Q&A Mejorado con Recuperación de Información (RAG)
# Uso de LangChain con ChromaDB
#
import openai
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

# Configurar OpenAI API Key
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),  # This is the default and can be omitted
    #api_key="your-openai-api-key"
)
# Cargar la base de datos de documentos en ChromaDB
vectorstore = Chroma(persist_directory="./chroma_db", embedding_function=OpenAIEmbeddings())

# Crear un retriever para buscar información en la base de datos
retriever = vectorstore.as_retriever()

# Crear la Conversational Retrieval Chain con GPT-4
qa_chain = ConversationalRetrievalChain.from_llm(
    ChatOpenAI(model_name="gpt-4"),
    retriever=retriever
)

# Hacer una pregunta con recuperación de documentos
query = "¿Qué es LangChain?"
response = qa_chain({"question": query})
print(response["answer"])


ValueError: Missing some input keys: {'chat_history'}